In [ ]:
#Importing the libraries
import cv2 
from ultralytics import YOLO
import time
import requests

In [ ]:
# Load YOLO11n model
model = YOLO('runs/model/best.pt')

In [ ]:
# Telegram Bot Details
BOT_TOKEN = "8188314211:AAHSYtQZJl57XXCMF52BLSJIrLzdFfz7KhQ"  # Replace with your Telegram bot token
CHAT_ID = "1443202982"  # Replace with your Telegram chat ID

In [ ]:
#Telegram related functions for message delivery
def send_telegram_alert(message):
    """Send a Telegram message"""
    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage"
    payload = {"chat_id": CHAT_ID, "text": message}
    response = requests.post(url, json=payload)
    print("✅ Telegram alert sent!", response.json())

In [ ]:
# Load video
video_path = "test videos/tomato_video.mp4"  # Change this to your video file
output_path = "test and output data/output videos/output_video_2.mp4"

In [ ]:
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Get FPS to calculate time

In [ ]:
# Video writer to save output
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

In [ ]:
low_count_start_time = None  # Time when tomatoes go below 5
alert_sent = False  # To track if alert has been sent

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("❌ Error: Frame could not be read. Video might be corrupted or incompatible.")
        send_telegram_alert("❌ Error: Frame could not be read. Video might be corrupted or incompatible.")
        break

    # Run YOLOv8 detection
    results = model(frame)

    # Count tomatoes (Assuming class 0 = Tomato)
    tomato_count = 0
    for r in results:
        for box in r.boxes:
            class_id = int(box.cls[0])  # Class index
            confidence = float(box.conf[0])  # Confidence score
            if class_id == 0 and confidence >= 0.3:  # Filter tomatoes
                tomato_count += 1
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, "Tomato", (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display count on video
    cv2.putText(frame, f"Tomatoes: {tomato_count}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Save frame to video
    out.write(frame)

    # Check if tomato count is less than 5
    if tomato_count < 5:
        if low_count_start_time is None:  
            low_count_start_time = time.time()  # Start timer
        elif time.time() - low_count_start_time >= 5 and not alert_sent:  
            send_telegram_alert(f"⚠️ Low Tomato Alert! Only {tomato_count} detected for 5+ seconds.")
            alert_sent = True  # Prevent multiple alerts
    else:
        low_count_start_time = None  # Reset timer if count is back to normal
        alert_sent = False  # Allow future alerts

    # Show frame (optional)
    cv2.imshow("Tomato Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Processed video saved as {output_path}")